In [17]:
#Imports
import requests as rq
import json
import requests
import urllib2
from bs4 import BeautifulSoup
import re
import string
from afinn import Afinn
import os
import matplotlib.pyplot as plt


In [18]:
#rapgenius information
rap_genius_id = "_17NJOxR9_RGp1XTQ1Vl3O--bxYh-cSzcCwASKlr-Xlyj3vh2_BsdedBZLX6xgdt"
client_secret = "2wlydcjFiqOlSI5Zi65OeizvaWRx16xkkjIBfPkqZ9t1aFNNYTrjza8ny6xYlXdJQSP_vMzj6e20O7ORtjIIcQ"
client_access_token = "Uzv_tj6uOXE9_1eX2wCrOGi3j4JtmYcm1kVF-UmPcC5gJh1MQZKJLph4aFp9frFx"

In [19]:
def getLyrics(URL):
    page = requests.get(URL)
    html = BeautifulSoup(page.text, "html.parser") # Extract the page's HTML as a string

    # Scrape the song lyrics from the HTML
    lyrics = html.find("div", class_="lyrics").get_text()
    return lyrics
    

***GETTING NEGATIVE WORDS***

dictionary of words is score_word_dict

In [20]:
#Get requests response "Failed to establish a new connection: [Errno -2]"
url_web = "https://raw.githubusercontent.com/hyharryhuang/AFINN-json/master/AFINN.json"
#We are going to hardcode the Afinn-111 json

afinn_data = rq.get(url_web).json()

In [21]:
score_word_dict = dict(afinn_data) #negative word dictionary

In [22]:
#add firearms
firearm_url = "https://genius.com/Rap-genius-firearms-and-firearm-companies-mentioned-in-rap-lyrics"
scrape_words = getLyrics(firearm_url)
firearm_list = scrape_words[130:].split("\n")

#add firearms in dictionary with sentiment score of -4
for firearm in firearm_list:
    score_word_dict[firearm.lower()] = -4

In [23]:
#add bulleted lists on rapgenius to a list that is returned and adds to score_word_dict
#needs score_word_dict to word, and adds the sentiment of that word (number)
def add_bulleted_lists(URL, number):
    words_drug = getLyrics(URL)
    first_list = words_drug.split("\n")
    new_list = []

    for x in first_list:
        new_list.append(x.replace(u'\u2022 ', ''))

    #add drugs in dictionary with sentiment score of -1
    for thing in new_list:
        score_word_dict[thing.lower()] = number

In [24]:
slang_drugs_url = "https://genius.com/Rap-genius-slang-dictionary-drugs-lyrics"
slang_firearms_url = "https://genius.com/Rap-genius-slang-dictionary-firearms-lyrics"

#adds words into sentiment dictionary
add_bulleted_lists(slang_drugs_url, -2)
add_bulleted_lists(slang_firearms_url, -4)

***REQUEST RAPGENIUS API***

***GET RAPGENIUS LYRICS***

use: get_song_lyrics(artist, song name)

In [27]:
#input: artist and track title as string
#output: lyrics of song track as string
def song_lyrics(artist, track_title):
    track_title = track_title.replace("&", "and").partition("(")[0].strip()
    def _process(text):
        # formats the url
        puncs = set(string.punctuation) - set("-")
        text = [''.join([c for c in s if c not in puncs]).lower() 
                  for s in unidecode.unidecode(text).split()]
        return text
        
    artist = "-".join(_process(artist)).capitalize()
    track_title = "-".join(_process(track_title))
    
    URL = "https://genius.com/{'-'.join([artist, track_title])}-lyrics"
    
    # request page to get markup
    page = rq.get(URL)
    html = BeautifulSoup(page.text, "html.parser")
    lyrics = html.find("div", class_="lyrics").get_text()

    lyrics = " ".join([line for line in lyrics.strip("\n").split("\n") if not line.startswith('[') and line != ""])
    
    return " ".join(_process(lyrics))

***ANALYZE SENTIMENT SCORE FOR LYRICS***

In [47]:
#input: none
#output: list of rappers
def get_rappers():
    
    def validate(name):
        # checks if the name has a
        illegal_chars = "|?:^*\""

        for c in illegal_chars:
            if c == '|':
                if name.find(c) > 0:
                    name = name.split("|")[1]
            if c == "\"":
                if name.find(c) > 0:
                    name = name.replace(r'"','',2)
            else: 
                name = name.replace(c,'')
        return name
    
    with open("data/rappers.txt", "r") as f:
        rappers = f.read()
        
    return [validate(rapper).strip() for rapper in rappers.split("\n")]

x = get_rappers() 

In [35]:
g = open("data/rappers.txt", "r")
info = g.read()
x = info.split("\n")

In [37]:
#input: song as lyrics as string
#output: sentiment score as int
def sentimentAnalysis(song):
    lyrics_list = song.split(" ")
    value = 0
    senti = 0
    for word in lyrics_list:
        if word in score_word_dict:
            value = value + int(score_word_dict[word])
        else:
            continue
        senti = value / (1.0 * len(lyrics_list))
    return senti

In [50]:
#input: artist name as string
#output: lyrics in artist discography as one string
def get_bf1(artist):
    with open("data/lyrics/%s.txt" % artist, "r") as f:
        info = eval(f.read())
    big_string = ""
    for album in info:
        for track in album["tracks"]:
            big_string += track["lyrics"]
    return big_string

In [39]:
with open("data/done.txt", "r") as t:
    info = t.read()
done_artist = info.split("\n")

In [ ]:
with open("data/dictionary.txt", 'r', encoding='utf8') as f:
    score_word_dict = eval(f.read())

In [70]:
#running sentiment analysis on rappers

dict_senti = {}
for rapper in x:
    if rapper in done_artist:
        continue
    else: 
        print(rapper)
        try:
            dict_senti[rapper] = sentimentAnalysis(get_bf1(rapper))
        except (SyntaxError, TypeError):
            continue
    

03 Greedo
100 Kila
100s
12 Gauge
2 Chainz
2 Pistols
2$ Fabo
21 Savage
2Mex
360
40 Glocc
50 Cent
6ix9ine
6lack
The 6th Letter
9th Wonder
A+
A-Q
Ab-Soul
A Boogie wit da Hoodie
Abstract Rude
Ace Hood
Aceyalone
Action Bronson
Adam Saleh
Aesop Rock
Afrika Bambaataa
Afroman
Afu-Ra
Agallah
Ahmad Lewis
Ajs Nigrutin
AKA
Akala
Akinyele
Akir
Akon
The Alchemist
Alex Wiley
Ali Vegas
AMG
Amil
Ampichino
Anderson .Paak
Andre Nickatina
Andy Mineo
Angel Haze
Angie Martinez
Ant Banks
Anthony Parker
Antoinette
Anybody Killa
Apache
Apathy
Arabian Prince
Arin Hanson
A$AP Ferg
A$AP Nast
A$AP Rocky
A$AP Yams
A$ton Matthews
Asher Roth
Astronautalis
Awol One
Awkwafina
AZ
Azealia Banks
B-Legit
B-Real
B.G.
B.G. Knocc Out
B.o.B
Baby Bash
Baby Boy da Prince
Baby D
Bad Azz
Badshah
Baeza
Bahamadia
Baka Not Nice
Bang Yong-guk
Bangladesh
Bas
Battlecat
Beanie Sigel
Becky G
Benny Blanco
Beenzino
Benzino
BewhY
Bhad Bhabie
Big Boi
Big Daddy Kane
Big Ed
Big Gipp
Big Hawk
Big K.R.I.T.
Big Kuntry King
Big L
Big Lurch
Big Mell

IOError: [Errno 2] No such file or directory: "data/lyrics/Lil' Cease.txt"

In [75]:
#overwrite new senti dictionary
f = open("data/senti.txt", "w")
f.write(str(dict_senti))
f.close()

In [79]:
#add to done list of artists
g = open("data/done.txt", "w")
for key, value in dict_senti.iteritems():
    g.write(key)
    g.write("\n")
g.close()

In [ ]:
dict_senti

removelist = []
for key, value in dict_senti.iteritems():
    if value == -4:
        removelist.append(key)
    
for artist in removelist:
    del dict_senti[artist]

In [ ]:
f = open("data/senti.txt","w")
f.write(str(dict_senti))
f.close()

In [80]:
!git pull origin master

Updating acec925..c534609
Fast-forward
 create_network.ipynb                             | 220 ++---------------------
 data/networks/.json                              |   2 +-
 preliminary aggresion network with node size.png | Bin 0 -> 128981 bytes
 rappers_network_sentiment2.json                  |   2 +-
 4 files changed, 15 insertions(+), 209 deletions(-)
 create mode 100644 preliminary aggresion network with node size.png


From https://github.com/allengueco/rap_aggression
 * branch            master     -> FETCH_HEAD
   acec925..c534609  master     -> origin/master


***DATA STRUCTURED STORAGE (OPTIONAL)***